# Import Dependencies

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

# Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic  #Holistic Model
mp_drawing = mp.solutions.drawing_utils  #Drawing Utilites

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [9]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [15]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [14]:
# image_bgr = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

In [10]:
plt.imshow(image)

NameError: name 'image' is not defined

# Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x , res.y , res.z , res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x , res.y , res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x , res.y , res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x , res.y , res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Setup Folders for Collection

In [6]:
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

#Actions that we try to detect
actions = np.array(['Hello' ,'I Love You', 'Thanks' , 'Please'])

#Fifty videos per action worth of data
no_sequences = 30

#Each video are going to be 30 frames in length
sequence_length = 30

In [7]:
for action in actions:
    # Create the base directory if it does not exist
    os.makedirs(os.path.join(DATA_PATH, action), exist_ok=True)
    
    dirmax = 0  # Initialize dirmax to 0 if no subdirectories are present initially
    
    # Check if subdirectories exist, update dirmax accordingly
    subdirs = os.listdir(os.path.join(DATA_PATH, action))
    if subdirs:
        dirmax = np.max(np.array(subdirs).astype(int))
    
    for sequence in range(0,no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax + sequence)))
        except Exception as e:
            print(f"Error creating directory: {e}")

Error creating directory: [WinError 183] Cannot create a file when that file already exists: 'MP_Data\\Hello\\29'
Error creating directory: [WinError 183] Cannot create a file when that file already exists: 'MP_Data\\I Love You\\29'
Error creating directory: [WinError 183] Cannot create a file when that file already exists: 'MP_Data\\Thanks\\29'
Error creating directory: [WinError 183] Cannot create a file when that file already exists: 'MP_Data\\Please\\29'


# Collect keypoints Values for Training and Testing

In [16]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Loop through actions
    for action in actions:
        #Loop through sequences aka videos
        for sequence in  range(no_sequences):
            #Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)
                # Draw landmarks
                draw_styled_landmarks(image, results)
                #Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION...', (145,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (245,66,230), 4, cv2.LINE_AA)
                    
                    cv2.rectangle(image , (0 ,0),(640 , 30) , (0,0,0),-1)
                    cv2.putText(image, 'Collecting frames for {}: Video Number {}'.format(action, sequence), (15,18), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    cv2.addWeighted(image, 0.3,image,1-0.3,0,image)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.rectangle(image , (0 ,0),(640 , 30) , (0,0,0),-1)
                    cv2.putText(image, 'Collecting frames for {}: Video Number {}'.format(action, sequence), (15,18), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    cv2.addWeighted(image, 0.3,image,1-0.3,0,image)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                #Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [9]:
np.load('MP_Data\\Hello\\0\\0.npy')

array([ 0.53711939,  0.49314049, -1.07114661, ...,  0.        ,
        0.        ,  0.        ])

In [8]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

# Preprocess Data and Create Label and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

In [11]:
label_map = {label:num for num,label in enumerate(actions)}

In [12]:
label_map

{'Hello': 0, 'I Love You': 1, 'Thanks': 2, 'Please': 3}

In [121]:
label_map['Hello']

0

In [13]:
sequences , labels = [] , []
for action in actions:
    for sequence in range(no_sequences):
        window =[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

KeyboardInterrupt: 

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(120, 30, 1662)

In [16]:
x = np.array(sequences)

In [23]:
x.shape

(210, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)

In [90]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [52]:
y.shape

(210, 7)

In [18]:
x_train , x_test , y_train , y_test = train_test_split(x, y, test_size=0.05)

In [72]:
x_train.shape

(171, 30, 1662)

In [73]:
x_test.shape

(9, 30, 1662)

In [56]:
y_train.shape

(199, 7)

In [57]:
y_test.shape

(11, 7)

# Build and Train LSTM Neural Network

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True , activation='relu'))
model.add(LSTM(64, return_sequences=False , activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


4/4 [==============================] - 11s 91ms/step - loss: 2.1169 - categorical_accuracy: 0.2368
Epoch 2/2000
4/4 [==============================] - 0s 64ms/step - loss: 2.3802 - categorical_accuracy: 0.3070
Epoch 3/2000
4/4 [==============================] - 0s 61ms/step - loss: 1.3805 - categorical_accuracy: 0.3333
Epoch 4/2000
4/4 [==============================] - 0s 67ms/step - loss: 1.3849 - categorical_accuracy: 0.2018
Epoch 5/2000
4/4 [==============================] - 0s 67ms/step - loss: 1.3414 - categorical_accuracy: 0.2632
Epoch 6/2000
4/4 [==============================] - 0s 68ms/step - loss: 1.3207 - categorical_accuracy: 0.2807
Epoch 7/2000
4/4 [==============================] - 0s 80ms/step - loss: 1.3447 - categorical_accuracy: 0.4035
Epoch 8/2000
4/4 [==============================] - 0s 59ms/step - loss: 1.4067 - categorical_accuracy: 0.3947
Epoch 9/2000
4/4 [==============================] - 0s 73ms/step - loss: 1.3900 - categorical_accuracy: 0.280

In [129]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_19 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_20 (LSTM)              (None, 64)                49408     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 4)                 132       
                                                                 
Total params: 596708 (2.28 MB)
Trainable params: 59670

In [130]:
import visualkeras
from PIL import ImageFont
font = ImageFont.truetype("arial.ttf", 12)
visualkeras.layered_view(model, legend=True, font=font,spacing=100)

TypeError: 'int' object is not iterable

# Make Predictions

In [131]:
res = model.predict(x_test)

1/1 [==============================] - 1s 558ms/step


In [132]:
actions[np.argmax(res[4])]

'I Love You'

In [133]:
actions[np.argmax(y_test[4])]

'I Love You'

# Save Weights

In [134]:
model.save('action.h5')

c:\Users\deepa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
del model

In [41]:
model.load_weights('action.h5')

# Evaluation using Confusion Matrix and Accuracy

In [136]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [137]:
yhat = model.predict(x_train)

4/4 [==============================] - 0s 19ms/step


In [138]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [139]:
yhat

[3,
 1,
 2,
 1,
 3,
 3,
 0,
 3,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 3,
 3,
 0,
 2,
 3,
 2,
 3,
 3,
 1,
 2,
 3,
 0,
 1,
 3,
 3,
 3,
 1,
 3,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 0,
 1,
 1,
 3,
 2,
 3,
 1,
 1,
 0,
 3,
 3,
 1,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 3,
 1,
 0,
 0,
 2,
 3,
 2,
 1,
 0,
 2,
 0,
 3,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 3,
 1,
 3,
 2,
 0,
 0,
 3,
 2,
 1,
 0,
 3,
 2,
 3,
 1,
 0,
 3,
 1,
 2,
 0,
 3,
 2,
 1,
 2,
 2,
 1]

In [140]:
ytrue

[3,
 1,
 2,
 1,
 3,
 3,
 0,
 3,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 3,
 3,
 0,
 2,
 3,
 2,
 3,
 3,
 1,
 2,
 3,
 0,
 1,
 3,
 3,
 3,
 1,
 3,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 0,
 1,
 1,
 3,
 2,
 3,
 1,
 1,
 0,
 3,
 3,
 1,
 1,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 3,
 1,
 0,
 0,
 2,
 3,
 2,
 1,
 0,
 2,
 0,
 3,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 3,
 1,
 3,
 2,
 0,
 0,
 3,
 2,
 1,
 0,
 3,
 2,
 3,
 1,
 0,
 3,
 1,
 2,
 0,
 3,
 2,
 1,
 2,
 2,
 1]

In [141]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[86,  0],
        [ 0, 28]],

       [[85,  0],
        [ 0, 29]],

       [[86,  0],
        [ 0, 28]],

       [[85,  0],
        [ 0, 29]]], dtype=int64)

In [142]:
accuracy_score(ytrue, yhat)

1.0

# Test in Real Time

In [22]:
colors = [(245,117,16), (117,245,16), (16,117,245), (245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    if res is not None:
        for num, prob in enumerate(res):
            cv2.rectangle(output_frame, (0,60+num*40), (int(prob*150) , 90+num*40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0,85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

In [ ]:
plt.imshow(show(res, actions, image, colors))

In [25]:
# Detection Variables
sequence = []
sentence = []
predictions = []
threshold = 0.8
predicting = False  # Flag to control prediction

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Prediction Logic only when predicting is True
        if predicting:
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            res = None

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))

            # Viz Logic
            if res is not None and res[np.argmax(res)] > threshold:

                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3:
                sentence = sentence[-3:]

            image = prob_viz(res, actions, image, colors)

        # Display prediction status on the frame
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, '    '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA )


        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Check for key presses
        key = cv2.waitKey(10)
        if key & 0xFF == ord('q'):
            break
        elif key & 0xFF == ord('p'):  # Press 'p' to start/stop predictions
            predicting = not predicting

    cap.release()
    cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [26]:
cap.release()
cv2.destroyAllWindows()